In [1]:
import xmitgcm as xm
import scipy.io as sio
import numpy as np
import pandas as pd

This is how we transform the mitgcm velocity data into a ".mat" file. Note that the units of x,y,t need to match the units of U, V. If x, y are given in degrees and t is given in days. Then the velocity field needs to be given in deg/day. In case the velocity field ("U","V") is given in m/s, the units of "x","y" must be given in degrees and the units of "t". In case you need to convert the units of the velocity field to deg/day please use the functions [convert_meters_per_second_to_deg_per_day](convert_meters_per_second_to_deg_per_day.ipynb). This is important!

In [2]:
mitgcm_results_folder = r"D:\MITgcm_results_Leman_Sep2021\results"
mitgcm_ref_date= np.datetime64('2021-07-26T00:00:00')

date_start = np.datetime64('2021-08-30')
date_end = np.datetime64('2021-09-11')

index_depth = 0

In [3]:
str_ref_date = pd.Timestamp(mitgcm_ref_date).strftime('%Y-%m-%d %H:%M:%S').replace('00', '0')

In [4]:
ds_mitgcm = xm.open_mdsdataset(mitgcm_results_folder, ref_date=str_ref_date, delta_t=4, endian='<')

In [5]:
sliced_ds = ds_mitgcm.sel(time =slice(date_start, date_end), Z = ds_mitgcm['Z'].values[index_depth])

In [6]:
sliced_ds['XC'].values

array([2.5000e+01, 7.5000e+01, 1.2500e+02, ..., 6.7075e+04, 6.7125e+04,
       6.7175e+04], dtype=float32)

In [7]:
# Coordinates of the centers of the cells in meters
x = sliced_ds['XC'].values
y = sliced_ds['YC'].values 

In [8]:
u = sliced_ds['UVEL'].transpose('YC', 'XG', 'time').values
v = sliced_ds['VVEL'].transpose('YG', 'XC', 'time').values

In [9]:
t = (sliced_ds['time'].values - mitgcm_ref_date).astype('timedelta64[s]').astype(int)

In [10]:
# velocties in m/s
# time in number of seconds since time_ref 

data_dict = {
    'u': u, # array (NY, NX, NT)
    'v': v, # array (NY, NX, NT)
    'x': x, # array (NY, NX)
    'y': y, # array (NY, NX)
    't': t # array (1, NT)
}

# Save the data to a .mat file
sio.savemat(f'Leman_{date_start}_{date_end}.mat', data_dict)